In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Coronary_artery_disease"
cohort = "GSE59867"

# Input paths
in_trait_dir = "../../input/GEO/Coronary_artery_disease"
in_cohort_dir = "../../input/GEO/Coronary_artery_disease/GSE59867"

# Output paths
out_data_file = "../../output/preprocess/Coronary_artery_disease/GSE59867.csv"
out_gene_data_file = "../../output/preprocess/Coronary_artery_disease/gene_data/GSE59867.csv"
out_clinical_data_file = "../../output/preprocess/Coronary_artery_disease/clinical_data/GSE59867.csv"
json_path = "../../output/preprocess/Coronary_artery_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling reveals potential prognostic biomarkers associated with the progression of heart failure"
!Series_summary	"Heart failure (HF) is the most common cause of morbidity and mortality in the developed countries, especially considering the present demographic tendencies in those populations."
!Series_summary	"We identified biologically relevant transcripts that are significantly altered in the early phase of myocardial infarction (MI) and are associated with the development of post-myocardial infarction HF."
!Series_overall_design	"We collected peripheral blood samples from patients (n=111) with ST-segment elevation myocardial infarction (STEMI) at four time points (admission, discharge, 1 month after MI, and 6 months after MI). Control group comprised patients (n=46) with a stable coronary artery disease and without a history of myocardial infarction. Affymetrix HuGene 1.0 ST arrays were used to analyze mRNA levels in periperal

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from PBMCs
# using Affymetrix HuGene 1.0 ST arrays, so it's suitable for our analysis.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the Sample Characteristics Dictionary, we can identify the following:

# For trait (Coronary_artery_disease):
# Row 2 contains 'hf progression', which has information about heart failure (HF) and stable CAD
trait_row = 2

# Age is not explicitly mentioned in the characteristics
age_row = None

# Gender is not explicitly mentioned in the characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert trait value to binary:
    1 for MI patients (HF, non-HF) - these are the cases with CAD that had myocardial infarction
    0 for controls (stable CAD without MI history)
    None for unclear cases
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Print for debugging
    print(f"Processing trait value: '{value}'")
    
    if value == 'stable CAD':
        return 0  # Control group (CAD without MI)
    elif value in ['HF', 'non-HF']:
        return 1  # CAD/MI patients (cases)
    elif value == 'N/A':
        return None
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion since age data is not available"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion since gender data is not available"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Print the unique values in the trait row for debugging
    print("Unique values in trait_row:")
    unique_vals = clinical_data.iloc[trait_row].unique()
    for val in unique_vals:
        print(f"  {val}")
    
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Unique values in trait_row:
  !Sample_characteristics_ch1
  hf progression: N/A
  hf progression: HF
  hf progression: non-HF
  hf progression: stable CAD
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'HF'
Processing trait value: 'HF'
Processing trait value: 'HF'
Processing trait value: 'HF'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processing trait value: 'N/A'
Processin

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Coronary_artery_disease/GSE59867/GSE59867_family.soft.gz
Matrix file: ../../input/GEO/Coronary_artery_disease/GSE59867/GSE59867_series_matrix.txt.gz
Found the matrix table marker at line 66


Gene data shape: (33297, 436)
First 20 gene/probe identifiers:
['7892501', '7892502', '7892503', '7892504', '7892505', '7892506', '7892507', '7892508', '7892509', '7892510', '7892511', '7892512', '7892513', '7892514', '7892515', '7892516', '7892517', '7892518', '7892519', '7892520']


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the expression data appear to be numerical probe IDs (7892501, 7892502, etc.)
# These are not standard human gene symbols (which would be like BRCA1, TP53, etc.)
# Most likely these are Illumina or Affymetrix probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Examine the gene_assignment column that contains gene symbol information
print("\nExamining gene_assignment column format (first 3 rows):")
if 'ID' in gene_annotation.columns and 'gene_assignment' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: ID={gene_annotation['ID'].iloc[i]}")
        print(f"Gene assignment: {gene_annotation['gene_assignment'].iloc[i][:200]}...")  # Show first 200 chars

    # Check the quality and completeness of the mapping
    non_null_assignments = gene_annotation['gene_assignment'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nGene assignment column completeness: {non_null_assignments}/{total_rows} rows ({non_null_assignments/total_rows:.2%})")
    
    # Check if some extracted gene symbols can be found in the gene_assignment column
    print("\nAttempting to extract gene symbols from the first few gene_assignment entries:")
    for i in range(min(3, len(gene_annotation))):
        if pd.notna(gene_annotation['gene_assignment'].iloc[i]):
            symbols = extract_human_gene_symbols(str(gene_annotation['gene_assignment'].iloc[i]))
            print(f"Row {i} extracted symbols: {symbols}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['7896736', '7896738', '7896740'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008'], 'seqname': ['chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091'], 'RANGE_STOP': ['54936', '63887', '70008'], 'total_probes': [7.0, 31.0, 24.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST00000492842 // OR4G11P // olfactory receptor, family 4, subfamily G, member 11 pseudogene // --- // --- /// ENST00000588632 // OR4G1P // olfactory receptor, family 4, subf

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the gene annotation that contain probe IDs and gene symbols
# Based on examining the data, 'ID' appears to contain the probe IDs matching those in gene_data
# 'gene_assignment' contains the gene symbol information that needs extraction

# 2. Extract gene mapping dataframe from annotation
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Print mapping info to verify
print(f"Gene mapping dataframe shape: {mapping_data.shape}")
print("Sample of mapping data:")
print(mapping_data.head(3))

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# The apply_gene_mapping function will handle many-to-many mappings as specified
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_data)

# Normalize gene symbols to standard format
gene_data = normalize_gene_symbols_in_index(gene_data)

print(f"Gene expression data after mapping - shape: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Save gene data to CSV for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (33297, 2)
Sample of mapping data:
        ID                                               Gene
0  7896736                                                ---
1  7896738  ENST00000328113 // OR4G2P // olfactory recepto...
2  7896740  NM_001004195 // OR4F4 // olfactory receptor, f...


Gene expression data after mapping - shape: (24229, 436)
First 10 gene symbols after mapping:
['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Gene expression data saved to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE59867.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Attempt to load gene data and handle possible issues with normalization
try:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Check if gene_data (from previous step) has any content
    if gene_data.shape[0] == 0:
        print("WARNING: Gene data is empty after normalization in previous step.")
        print("This appears to be miRNA data rather than gene expression data.")
        
        # Since gene_data is empty, set gene_available to False
        is_gene_available = False
        
        # Create an empty dataframe for metadata purposes
        empty_df = pd.DataFrame()
        
        # Log information about this dataset for future reference
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Consider it biased as we can't use it
            df=empty_df,
            note="Dataset appears to contain miRNA data rather than gene expression data. Gene symbols could not be normalized."
        )
        
        print("Dataset marked as unusable due to lack of valid gene expression data.")
    else:
        # If gene_data is not empty, proceed with normalization and linking
        print(f"Gene data shape before normalization: {gene_data.shape}")
        
        # Save the gene data we have, even if it's already normalized
        gene_data.to_csv(out_gene_data_file)
        print(f"Gene data saved to {out_gene_data_file}")
        
        # Attempt to link clinical and gene data
        if is_trait_available:
            # Load clinical data
            clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
            print(f"Loaded clinical data shape: {clinical_features.shape}")
            
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate data quality and save cohort info
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note="Successfully processed gene expression data for coronary artery disease."
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            # Cannot proceed with linking if trait data is missing
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="Cannot link data because trait information is not available."
            )
except Exception as e:
    print(f"Error in data processing: {e}")
    
    # Log the error and mark the dataset as unusable
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # Consider gene data unavailable if we had an error
        is_trait_available=is_trait_available,
        is_biased=True,  # Consider it biased as we can't use it
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note=f"Error during normalization or linking: {str(e)}"
    )

Gene data shape before normalization: (24229, 436)


Gene data saved to ../../output/preprocess/Coronary_artery_disease/gene_data/GSE59867.csv
Loaded clinical data shape: (1, 436)
Initial linked data shape: (436, 24230)


Linked data shape after handling missing values: (110, 24230)
For the feature 'Coronary_artery_disease', the least common label is '0.0' with 46 occurrences. This represents 41.82% of the dataset.
The distribution of the feature 'Coronary_artery_disease' in this dataset is fine.



Linked data saved to ../../output/preprocess/Coronary_artery_disease/GSE59867.csv
